In [1]:
!pip install -U textblob
!pip install --upgrade pip
!pip install xgboost
!pip install wordcloud
!pip install numpy
!pip install pillow
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')
from textblob import TextBlob
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



  Using cached pip-24.2-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-24.2-py3-none-any.whl (1.8 MB)


ERROR: To modify pip, please run the following command:
C:\Users\ADMIN\anaconda3\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
df = pd.read_csv('recession_2023_india_tweets_dataset.csv')

In [3]:
df.head()

,date,id,content,username,likeCount,retweetCount
0,2022-11-30 04:17:49+00:00,1597807084934172672,📌 @elonmusk begins hiring for Twitter 2.0 afte...,workflexitweets,1,1
1,2022-11-30 02:48:10+00:00,1597784522728558592,"Global Recession से दुनिया हुई बेहाल, लेकिन In...",indilinks,0,0
2,2022-11-29 17:21:57+00:00,1597642029479038976,Will global recession impact India or not? #re...,BizzBuzzNews,0,0
3,2022-11-29 11:40:40+00:00,1597556142213070850,Is Recession In India Coming Soon?\n\nWant to ...,Anubhav74782862,0,0
4,2022-11-29 10:12:57+00:00,1597534071202385922,A report by #realestate data collation &amp; r...,indiaoutbound,1,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5176 entries, 0 to 5175
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          5176 non-null   object
 1   id            5176 non-null   int64 
 2   content       5176 non-null   object
 3   username      5176 non-null   object
 4   likeCount     5176 non-null   int64 
 5   retweetCount  5176 non-null   int64 
dtypes: int64(3), object(3)
memory usage: 242.8+ KB


In [5]:
df.isnull().sum()

date            0
id              0
content         0
username        0
likeCount       0
retweetCount    0
dtype: int64

In [6]:
df.columns

Index(['date', 'id', 'content', 'username', 'likeCount', 'retweetCount'], dtype='object')

In [7]:
text_df = df.drop(['date', 'id', 'username', 'likeCount',
       'retweetCount'], axis=1)
text_df.head()

,content
0,📌 @elonmusk begins hiring for Twitter 2.0 afte...
1,"Global Recession से दुनिया हुई बेहाल, लेकिन In..."
2,Will global recession impact India or not? #re...
3,Is Recession In India Coming Soon?\n\nWant to ...
4,A report by #realestate data collation &amp; r...


In [8]:
print(text_df['content'].iloc[0],"\n")
print(text_df['content'].iloc[1],"\n")
print(text_df['content'].iloc[2],"\n")
print(text_df['content'].iloc[3],"\n")
print(text_df['content'].iloc[4],"\n")

📌 @elonmusk begins hiring for Twitter 2.0 after firing nearly 5000 employees, reveals future plans

📌 @amazonIN likely to fire hundreds of workers in India next month
#Hiring #ELONMUSK #layoffs #Amazon #recession #layoff #job 

Global Recession से दुनिया हुई बेहाल, लेकिन India ने कर डाला बड़ा कमाल, आखिर कैसे चला भारत का सिक्का
#global #recession #globalrecession2022
#indilinksnews

https://t.co/ghnwjoMsj5 https://t.co/own7FgFqGR 

Will global recession impact India or not? #recession #USA #layoffs #realestatesector https://t.co/wU5DKBul9g 

Is Recession In India Coming Soon?

Want to know read our article: 
https://t.co/Yq6hoqOjv5

#recession #India https://t.co/IrEdXvIvVX 

A report by #realestate data collation &amp; research firm #CoStar says that appetite for new investments in hotels in #Europe has continued to grow, despite #RussiaUkraineWar, record-high #inflation &amp; even signs of a #recession. #IO @CoStarUS @alvarezmarsal

https://t.co/0QDn2ujQUm 



In [9]:
text_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5176 entries, 0 to 5175
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   content  5176 non-null   object
dtypes: object(1)
memory usage: 40.6+ KB


In [10]:
def data_processing(text):
    text = text.lower()
    text = re.sub(r"https\S+|www\S+https\S+", '',text, flags=re.MULTILINE)
    text = re.sub(r'\@w+|\#','',text)
    text = re.sub(r'[^\w\s]','',text)
    text_tokens = word_tokenize(text)
    filtered_text = [w for w in text_tokens if not w in stop_words]
    return " ".join(filtered_text)

In [11]:
text_df.content = text_df['content'].apply(data_processing)
text_df = text_df.drop_duplicates('content')

In [12]:
stemmer = PorterStemmer()
def stemming(data):
    text = [stemmer.stem(word) for word in data]
    return data

In [ ]:
text_df['content'] = text_df['content'].apply(lambda x: stemming(x))

In [ ]:
text_df.head()

In [ ]:
print(text_df['content'].iloc[0],"\n")
print(text_df['content'].iloc[1],"\n")
print(text_df['content'].iloc[2],"\n")
print(text_df['content'].iloc[3],"\n")
print(text_df['content'].iloc[4],"\n")

In [ ]:
text_df.info()

In [ ]:
def polarity(text):
    return TextBlob(text).sentiment.polarity

In [ ]:
text_df['polarity'] = text_df['content'].apply(polarity)

In [ ]:
text_df.head(10)

In [ ]:
def sentiment(label):
    if label <0:
        return "Negative"
    elif label ==0:
        return "Neutral"
    elif label>0:
        return "Positive"

In [ ]:
text_df['sentiment'] = text_df['polarity'].apply(sentiment)

In [ ]:
text_df.head()

In [ ]:
fig = plt.figure(figsize=(5,5))
sns.countplot(x='sentiment', data = text_df)

In [ ]:
fig = plt.figure(figsize=(7,7))
colors = ("yellowgreen", "gold", "red")
wp = {'linewidth':2, 'edgecolor':"black"}
tags = text_df['sentiment'].value_counts()
explode = (0.1,0.1,0.1)
tags.plot(kind='pie', autopct='%1.1f%%', shadow=True, colors = colors,
         startangle=90, wedgeprops = wp, explode = explode, label='')
plt.title('Distribution of sentiments')

In [ ]:
pos_tweets = text_df[text_df.sentiment == 'Positive']
pos_tweets = pos_tweets.sort_values(['polarity'], ascending= False)
pos_tweets.head()

In [ ]:
text = ' '.join([word for word in pos_tweets['content']])
plt.figure(figsize=(20,15), facecolor='None')
wordcloud = WordCloud(max_words=500, width=1600, height=800).generate(text)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.title('Most frequent words in positive tweets', fontsize=19)
plt.show()

In [ ]:
neg_tweets = text_df[text_df.sentiment == 'Negative']
neg_tweets = neg_tweets.sort_values(['polarity'], ascending= False)
neg_tweets.head()


In [ ]:
text = ' '.join([word for word in neg_tweets['content']])
plt.figure(figsize=(20,15), facecolor='None')
wordcloud = WordCloud(max_words=500, width=1600, height=800).generate(text)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.title('Most frequent words in negative tweets', fontsize=19)
plt.show()

In [ ]:
neutral_tweets = text_df[text_df.sentiment == 'Neutral']
neutral_tweets = neutral_tweets.sort_values(['polarity'], ascending= False)
neutral_tweets.head()

In [ ]:
text = ' '.join([word for word in neutral_tweets['content']])
plt.figure(figsize=(20,15), facecolor='None')
wordcloud = WordCloud(max_words=500, width=1600, height=800).generate(text)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.title('Most frequent words in neutral tweets', fontsize=19)
plt.show()

In [ ]:
vect = CountVectorizer(ngram_range=(1,2)).fit(text_df['content'])

In [ ]:
feature_names = vect.get_feature_names_out()
print("Number of features: {}\n".format(len(feature_names)))
print("First 20 features:\n {}".format(feature_names[:20]))

In [ ]:
X = text_df['content']
Y = text_df['sentiment']
X = vect.transform(X)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
print("Size of x_train:", (x_train.shape))
print("Size of y_train:", (y_train.shape))
print("Size of x_test:", (x_test.shape))
print("Size of y_test:", (y_test.shape))

In [ ]:
#Logistic Regression
logreg = LogisticRegression()
logreg.fit(x_train, y_train)
logreg_pred = logreg.predict(x_test)
logreg_acc = accuracy_score(logreg_pred, y_test)
print("Test accuracy: {:.2f}%".format(logreg_acc*100))

In [ ]:
print(confusion_matrix(y_test, logreg_pred))
print("\n")
print(classification_report(y_test, logreg_pred))

In [ ]:
style.use('classic')
cm = confusion_matrix(y_test, logreg_pred, labels=logreg.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels=logreg.classes_)
disp.plot()

In [ ]:
# XGBoost
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)

xgb_model = xgb.XGBClassifier()
xgb_model.fit(x_train, y_train_encoded)

xgb_pred = xgb_model.predict(x_test)
xgb_acc = accuracy_score(y_test_encoded, xgb_pred)
print("Test accuracy: {:.2f}%".format(xgb_acc*100))

print(confusion_matrix(y_test_encoded, xgb_pred))
print("\n")
print(classification_report(y_test_encoded, xgb_pred))



In [ ]:
cm = confusion_matrix(y_test_encoded, xgb_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=le.classes_)
disp.plot()
plt.show()

In [ ]:
# Naive Bayes
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB

nb_model = MultinomialNB()
nb_model.fit(x_train, y_train)

nb_pred = nb_model.predict(x_test)
nb_acc = accuracy_score(nb_pred, y_test)
print("Test accuracy: {:.2f}%".format(nb_acc*100))

print(confusion_matrix(y_test, nb_pred))
print("\n")
print(classification_report(y_test, nb_pred))



In [ ]:
cm = confusion_matrix(y_test, nb_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=le.classes_)
disp.plot()
plt.show()

In [ ]:
# KNN
from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier()
knn_model.fit(x_train, y_train)

knn_pred = knn_model.predict(x_test)
knn_acc = accuracy_score(knn_pred, y_test)
print("Test accuracy: {:.2f}%".format(knn_acc*100))

print(confusion_matrix(y_test, knn_pred))
print("\n")
print(classification_report(y_test, knn_pred))



In [ ]:
cm = confusion_matrix(y_test, knn_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=le.classes_)
disp.plot()
plt.show()

In [ ]:

svm_model = svm.SVC()
svm_model.fit(x_train, y_train_encoded)

svm_pred = svm_model.predict(x_test)
svm_acc = accuracy_score(y_test_encoded, svm_pred)
print("Test accuracy: {:.2f}%".format(svm_acc*100))

print(confusion_matrix(y_test_encoded, svm_pred))
print("\n")
print(classification_report(y_test_encoded, svm_pred))



In [ ]:
cm = confusion_matrix(y_test_encoded, svm_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=le.classes_)
disp.plot()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay

# Define the models and their corresponding accuracy scores
models = [
    ('Logistic Regression', logreg_acc),
    ('XGBoost', xgb_acc),
    ('Naive Bayes', nb_acc),
    ('KNN', knn_acc),
    ('SVM', svm_acc)
]

# Create a dataframe to store the results
df = pd.DataFrame(models, columns=['Model', 'Accuracy'])

# Plot the results using a bar chart
plt.figure(figsize=(10, 6))
plt.bar(df['Model'], df['Accuracy'])
plt.xlabel('Model')
plt.ylabel('Accuracy')
plt.title('Comparative Analysis of Models')
plt.show()
#Table 
table = pd.DataFrame(df['Accuracy'].values, columns=['Accuracy'])
table.insert(0, 'Model', df['Model'])
print(table)